In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
print(sys.path)
from src.utils import *
from src.calcEMA import *
from src.robo import *
from src.myenv import *
import plotly.express as px

In [ ]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
# lightgbm  xgboost
estimator = 'lr'
_compare_models = False

start_train_date = '2023-06-01'  # train < and test >=
start_test_date = '2023-08-08'  # train < and test >=

stop_loss = 2.0
label = 'status'
regression_times = 24 * 30 * 2  # horas
times_regression_profit_and_loss = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close' ]
calc_rsi = True


### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [ ]:
use_cols = date_features + numeric_features
print('start_train_engine: use cols: ', use_cols)
print(f'start_train_engine: reading data - start date: {start_train_date}...')
all_data = get_data(symbol, save_database=False, interval='1h', tail=-1, columns=use_cols)
print('start_train_engine: get_data  all_data duplicated: ', all_data.index.duplicated().sum())

all_data = all_data[(all_data['open_time'] >= start_train_date)]  # .copy()
print('start_train_engine: info after reading data: ')
all_data.info()
print('start_train_engine: filter start_train_date all_data duplicated: ', all_data.index.duplicated().sum())

if calc_rsi:
    print('start_train_engine: calculating RSI...')
    all_data = calc_RSI(all_data)
    numeric_features.append('rsi')
    all_data.dropna(inplace=True)
print('start_train_engine: info after calculating RSI: ')
all_data.info()
print('start_train_engine: all_data duplicated: ', all_data.index.duplicated().sum())

print('start_train_engine: calculating regresstion_times...')
all_data, features_added = regresstion_times(all_data, numeric_features, regression_times, last_one=False)
print('start_train_engine: info after calculating regresstion_times: ')
all_data.info()
print('start_train_engine: all_data duplicated: ', all_data.index.duplicated().sum())

In [ ]:
print(f'start_train_engine: Filtering test_data: start_test_date: {start_test_date}')
test_data = all_data[all_data['open_time'] >= start_test_date]
print('start_train_engine: info after filtering test_data: ')
test_data.info()
print('start_train_engine: test_data duplicated: ', test_data.index.duplicated().sum())

In [ ]:
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
predict = experiment.predict_model(model, data=test_data)
predict = predict.sort_values(date_features)

In [ ]:
def test_trading_crypto_v3(data: pd.DataFrame, start_date, end_date, value: float, stop_loss = 3.0):
  _data = data.copy()
  _data.index = _data['open_time']
  _data = _data[(_data.index >= start_date) & (_data.index <= end_date)]
  saldo = value
  operacao = ''
  comprado = False
  valor_compra = 0 
  valor_venda = 0
  diff = 0.0
  
  operacao_compra = ''
  for row_nu in range(1, _data.shape[0]):
    open_time = pd.to_datetime(_data.iloc[row_nu:row_nu+1]['open_time'].values[0]).strftime("%Y-%m-%d %Hh")
    type(open_time)
    operacao = _data.iloc[row_nu:row_nu+1]['prediction_label'].values[0]
    #print(operacao)
    if (operacao.startswith('SOBE') or operacao.startswith('CAI')) and not comprado:
      operacao_compra = operacao
      valor_compra = round(_data.iloc[row_nu:row_nu+1]['close'].values[0], 2)
      print(f'[{row_nu}][{operacao_compra}][{open_time}] => Compra: {valor_compra}')
      comprado = True

    if comprado:
      diff = 100 * (_data.iloc[row_nu:row_nu+1]['close'].values[0] - valor_compra) / valor_compra
      #print(f'[{row_nu}][{operacao_compra}][{open_time}] Diff ==> {round(diff,2)}% - Comprado: {comprado}')
    
    if (abs(diff) >= stop_loss) and comprado:
      valor_venda = round(_data.iloc[row_nu:row_nu+1]['close'].values[0],2)
      if operacao_compra.startswith('SOBE'):
        saldo += round(saldo * (diff/100), 2)
      else:
        saldo += round(saldo * (-diff/100), 2)
      print(f'[{row_nu}][{operacao_compra}][{open_time}] => Venda: {valor_venda} => Diff: {round(diff,2)}% ==> Saldo: {saldo}')
      comprado = False

  print(f'Saldo: {saldo}')
  return saldo

print('Min Data: ', predict['open_time'].min())
print('Max Data: ', predict['open_time'].max())
test_trading_crypto_v3(predict, '2023-01-01', predict['open_time'].max(), 100.0, stop_loss)